<a href="https://colab.research.google.com/github/nizomjon9909/ExcelExamples/blob/master/Data_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Integration - Exercise
As we noticed, the data from the first part wasn't used in building the SQL database. Let's fix that. 😉

Our goal is to build a clean, well-structured database containing records related to real estate prices from the past 5 years.
Follow these steps:
1. Programmatically extract information related to the section content in the GUS API.

In [7]:
import pandas as pd
import requests
import json
import urllib.parse
import time

# 1. Setup Base URLs and Parameters
URL = "https://api-dbw.stat.gov.pl/api/1.1.0/"
API_endpoint = "variable/variable-data-section"

# Years: Last 5 years (Adjust as needed, e.g., 2019-2023)
years = [2019, 2020, 2021, 2022, 2023]
# Period IDs: Q1=270, Q2=271, Q3=272, Q4=273 (from Class Notebook)
period_ids = [270, 271, 272, 273]

# Base parameters dictionary
params = {
    "lang": "en",
    "id-zmienna": "309",   # Real estate prices
    "id-przekroj": "485",  # Section (Geographic/Voivodeships)
    "numer-strony": "0",
    "ile-na-stronie": 50   # 16 voivodeships fit easily in one page
}

data_frames = []

# 2. Loop through Years and Periods to fetch data
for year in years:
    params["id-rok"] = year
    for period in period_ids:
        params["id-okres"] = period

        # Construct URL
        full_url = URL + API_endpoint + "?" + urllib.parse.urlencode(params)
        print(f"Fetching: {year} - Period {period}")

        try:
            response = requests.get(full_url)
            if response.status_code == 200:
                json_data = response.json()
                # The actual data list is usually under the 'data' key
                temp_df = pd.DataFrame(json_data['data'])
                data_frames.append(temp_df)
            else:
                print(f"Error {response.status_code} for {full_url}")
        except Exception as e:
            print(f"Failed to fetch data: {e}")

        # Sleep to be polite to the API
        time.sleep(1)

# 3. Combine all data into one DataFrame
raw_df = pd.concat(data_frames, ignore_index=True)

# Display result
print(f"Total records downloaded: {len(raw_df)}")
raw_df.head()

Fetching: 2019 - Period 270
Fetching: 2019 - Period 271
Fetching: 2019 - Period 272
Fetching: 2019 - Period 273
Fetching: 2020 - Period 270
Fetching: 2020 - Period 271
Fetching: 2020 - Period 272
Fetching: 2020 - Period 273
Fetching: 2021 - Period 270
Fetching: 2021 - Period 271
Fetching: 2021 - Period 272
Fetching: 2021 - Period 273
Fetching: 2022 - Period 270
Fetching: 2022 - Period 271
Fetching: 2022 - Period 272
Fetching: 2022 - Period 273
Fetching: 2023 - Period 270
Fetching: 2023 - Period 271
Fetching: 2023 - Period 272
Fetching: 2023 - Period 273
Total records downloaded: 1000


,rownumber,id-zmienna,id-przekroj,id-wymiar-1,id-pozycja-1,id-wymiar-2,id-pozycja-2,id-okres,id-sposob-prezentacji-miara,id-daty,id-brak-wartosci,id-tajnosci,id-flaga,wartosc,precyzja
0,1,309,485,10,33617,423,4801795,270,180,2019,253,43,36,5508.0,2
1,2,309,485,10,33617,423,4801796,270,180,2019,253,43,36,4666.0,2
2,3,309,485,10,33617,423,4801797,270,180,2019,253,43,36,4996.0,2
3,4,309,485,10,33619,423,4801795,270,180,2019,253,43,36,6519.0,2
4,5,309,485,10,33619,423,4801796,270,180,2019,253,43,36,5894.0,2


2. Prepare the appropriate mapping — we want to include human-readable text information in our database.

In [8]:
# 1. Fetch the Mapping Dictionary for Section 485
mapping_endpoint = "variable/variable-section-position"
mapping_params = {
    "lang": "en",
    "id-przekroj": "485"
}

mapping_url = URL + mapping_endpoint + "?" + urllib.parse.urlencode(mapping_params)
response = requests.get(mapping_url)
mapping_data = response.json()

# 2. Create a clean mapping DataFrame
# The API usually returns a list of objects with 'id-pozycja' and 'nazwa' (name)
mapping_df = pd.DataFrame(mapping_data)

# Let's inspect column names to ensure we merge correctly
# Usually we map 'id-pozycja' from here to 'id-pozycja-2' in the raw_df
print("Mapping Columns:", mapping_df.columns)

# 3. Merge raw data with mapping
# Adjust 'left_on' if the column name in raw_df is different (e.g., 'id-pozycja-1')
# Based on section 485, the region ID is usually 'id-pozycja-2'
final_df = raw_df.merge(
    mapping_df[['id-pozycja', 'nazwa-pozycja']],
    left_on='id-pozycja-2',
    right_on='id-pozycja',
    how='left'
)

# 4. Clean up columns (rename for clarity)
final_df = final_df.rename(columns={'nazwa-pozycja': 'Voivodeship', 'wartosc': 'Price_PLN', 'id-daty': 'Year', 'id-okres': 'Period_ID'}) # Removed inplace=True and reassigned

# Select only relevant columns for the DB
clean_db_df = final_df[['Year', 'Period_ID', 'Voivodeship', 'Price_PLN']]
clean_db_df.head()

Mapping Columns: Index(['id-przekroj', 'id-wymiar', 'nazwa-wymiar', 'id-pozycja',
       'nazwa-pozycja'],
      dtype='object')


,Year,Period_ID,Voivodeship,Price_PLN
0,2019,270,new premises market,5508.0
1,2019,270,existing premises market,4666.0
2,2019,270,Total,4996.0
3,2019,270,new premises market,6519.0
4,2019,270,existing premises market,5894.0


3. Transfer the data from the API to the SQL database.

In [9]:
from sqlalchemy import create_engine

# 1. Create a SQLite database engine
# This will create a file named 'real_estate.db' in your current folder
engine = create_engine('sqlite:///real_estate.db')

# 2. Write the DataFrame to SQL
# 'real_estate_prices' will be the table name
clean_db_df.to_sql('real_estate_prices', con=engine, if_exists='replace', index=False)

print("Data successfully saved to SQL database 'real_estate.db' in table 'real_estate_prices'.")

# Optional: Verify by reading it back
# print(pd.read_sql("SELECT * FROM real_estate_prices LIMIT 5", con=engine))

Data successfully saved to SQL database 'real_estate.db' in table 'real_estate_prices'.
